In [95]:
import arxiv
import pandas as pd
import openai
from typing import Dict, List
import requests
from benlp.document import Document
import os
import pickle as pkl
from benlp.llms import embed_ada
import numpy as np

In [75]:
data_dir = "../data/arxiv-chat"
pdfs_dir = "../data/arxiv-chat/pdfs"

In [81]:
def get_articles(query, top_k=10):
    search = arxiv.Search(
        query=query,
        max_results=top_k,
        sort_by=arxiv.SortCriterion.Relevance,
    )
    results = list(search.results())
    return results


In [84]:
query = "Transformer machine learning"
articles = get_articles(query, top_k=10)

In [61]:
raw = [article._raw for article in articles]
raw[0]

{'id': 'http://arxiv.org/abs/2008.07758v1',
 'guidislink': True,
 'link': 'http://arxiv.org/abs/2008.07758v1',
 'updated': '2020-08-18T06:17:31Z',
 'updated_parsed': time.struct_time(tm_year=2020, tm_mon=8, tm_mday=18, tm_hour=6, tm_min=17, tm_sec=31, tm_wday=1, tm_yday=231, tm_isdst=0),
 'published': '2020-08-18T06:17:31Z',
 'published_parsed': time.struct_time(tm_year=2020, tm_mon=8, tm_mday=18, tm_hour=6, tm_min=17, tm_sec=31, tm_wday=1, tm_yday=231, tm_isdst=0),
 'title': 'Efficient Private Machine Learning by Differentiable Random\n  Transformations',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'http://export.arxiv.org/api/query?search_query=Transformer+machine+learning&id_list=&sortBy=relevance&sortOrder=descending&start=0&max_results=5',
  'value': 'Efficient Private Machine Learning by Differentiable Random\n  Transformations'},
 'summary': 'With the increasing demands for privacy protection, many privacy-preserving\nmachine learning systems were propo

In [85]:
for article in articles:
    # with open(f"{pdfs_dir}/{article.entry_id}.pdf", "wb") as f:
    #     f.write()
    article.download_pdf(pdfs_dir)

In [ ]:
base_fpath = "/Users/beneverman/Documents/Coding/benlp_v1/data/arxiv-chat/pdfs"
paths = [f"{base_fpath}/{path}" for path in os.listdir(base_fpath)]
docs = [Document(path).process() for path in paths]
doc_dicts = [doc.to_dict() for doc in docs]
with open(f"{data_dir}/transformers_10.pkl", "wb") as f:
    pkl.dump(doc_dicts, f)

In [91]:
with open(f"{data_dir}/transformers_10.pkl", "wb") as f:
    pkl.dump(doc_dicts, f)

In [92]:
with open(f"{data_dir}/transformers_10.pkl", "rb") as f:
    doc_dicts = pkl.load(f)

In [94]:
doc_dicts[0]['data']

[{'id': '118252-1',
  'text': '1\nLearning Filter Bank Sparsifying Transforms\nLuke Pﬁster, Student Member, IEEE, Yoram Bresler, Fellow, IEEE\nAbstract\nData is said to follow the transform (or analysis) sparsity model if it becomes sparse when acted on by a linear operator called\na sparsifying transform. Several algorithms have been designed to learn such a transform directly from data, and data-adaptive\nsparsifying transforms have demonstrated excellent performance in signal restoration tasks. Sparsifying transforms are typically\nlearned using small sub-regions of data called patches, but these algorithms often ignore redundant information shared between\nneighboring patches.\nWe show that many existing transform and analysis sparse representations can be viewed as ﬁlter banks, thus linking the\nlocal properties of patch-based model to the global properties of a convolutional model. We propose a new transform learning',
  'embedding': [-0.013519319705665112,
   0.00517689390107989

In [105]:
def order_by_similarity(query, docs, top_k=10):
    query_embedding = embed_ada(query)
    doc_scores = []
    for doc in docs:
        for page in doc['data']:
            score = np.dot(query_embedding, page['embedding'])
            doc_scores.append((doc['id'], page['id'], score))
    doc_scores = sorted(doc_scores, key=lambda x: x[2], reverse=True)
    return doc_scores[:top_k]

In [98]:
query = "What is the best transformer architecture?"


In [106]:
sorted_tuples = order_by_similarity(query, doc_dicts)

IndexError: list index out of range